In [1]:
import tensorflow as tf
import timeit
from datetime import datetime

https://www.tensorflow.org/guide/intro_to_graphs

You create and run a graph in TensorFlow by using tf.function, either as a direct call or as a decorator. tf.function takes a regular function as input and returns a Function. A Function is a Python callable that builds TensorFlow graphs from the Python function. You use a Function in the same way as its Python equivalent

`tf.function`

https://www.tensorflow.org/api_docs/python/tf/function

https://www.tensorflow.org/guide/function

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/eager/def_function.py

`tracing`

tensorflow.org/guide/function#tracing


`tf.autograph`

https://www.tensorflow.org/api_docs/python/tf/autograph

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/index.md


In [2]:
# Define a Python function.
def a_regular_function(x, y, b):
    x = tf.matmul(x, y)
    x = x + b
    return x

In [3]:
# `a_function_that_uses_a_graph` is a TensorFlow `Function`.
a_function_that_uses_a_graph = tf.function(a_regular_function)

In [4]:
# Make some tensors.
x1 = tf.constant([[1.0, 2.0]])
y1 = tf.constant([[2.0], [3.0]])
b1 = tf.constant(4.0)

In [5]:
orig_value = a_regular_function(x1, y1, b1).numpy()
# Call a `Function` like a Python function.
tf_function_value = a_function_that_uses_a_graph(x1, y1, b1).numpy()
assert(orig_value == tf_function_value)

In [6]:
def inner_function(x, y, b):
  x = tf.matmul(x, y)
  x = x + b
  return x

# Use the decorator to make `outer_function` a `Function`.
@tf.function
def outer_function(x):
  y = tf.constant([[2.0], [3.0]])
  b = tf.constant(4.0)

  return inner_function(x, y, b)

# Note that the callable will create a graph that
# includes `inner_function` as well as `outer_function`.
outer_function(tf.constant([[1.0, 2.0]])).numpy()

array([[12.]], dtype=float32)

Any function you write with TensorFlow will contain a mixture of built-in TF operations and Python logic, such as if-then clauses, loops, break, return, continue, and more. While TensorFlow operations are easily captured by a tf.Graph, Python-specific logic needs to undergo an extra step in order to become part of the graph. tf.function uses a library called AutoGraph (tf.autograph) to convert Python code into graph-generating code.

In [7]:
def simple_relu(x):
    if tf.greater(x, 0):
        return x
    else:
        return 0

In [8]:
# `tf_simple_relu` is a TensorFlow `Function` that wraps `simple_relu`.
tf_simple_relu = tf.function(simple_relu)

In [9]:
print("First branch, with graph:", tf_simple_relu(tf.constant(1)).numpy())
print("Second branch, with graph:", tf_simple_relu(tf.constant(-1)).numpy())

First branch, with graph: 1
Second branch, with graph: 0


A tf.Graph is specialized to a specific type of inputs (for example, tensors with a specific dtype or objects with the same id()).

Each time you invoke a Function with new dtypes and shapes in its arguments, Function creates a new tf.Graph for the new arguments. The dtypes and shapes of a tf.Graph's inputs are known as an input signature or just a signature.

The Function stores the tf.Graph corresponding to that signature in a ConcreteFunction. A ConcreteFunction is a wrapper around a tf.Graph

In [10]:
@tf.function
def my_relu(x):
  return tf.maximum(0., x)

# `my_relu` creates new graphs as it observes more signatures.
print(my_relu(tf.constant(5.5)))
print(my_relu([1, -1]))
print(my_relu(tf.constant([3., -3.])))

tf.Tensor(5.5, shape=(), dtype=float32)
tf.Tensor([1. 0.], shape=(2,), dtype=float32)
tf.Tensor([3. 0.], shape=(2,), dtype=float32)


In [11]:
# There are three `ConcreteFunction`s (one for each graph) in `my_relu`.
# The `ConcreteFunction` also knows the return type and shape!
print(my_relu.pretty_printed_concrete_signatures())

my_relu(x)
  Args:
    x: float32 Tensor, shape=(2,)
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x=[1, -1])
  Returns:
    float32 Tensor, shape=(2,)

my_relu(x)
  Args:
    x: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()


**The code in a Function can be executed both eagerly and as a graph. By default, Function executes its code as a graph:**

In [12]:
@tf.function
def get_MSE(y_true, y_pred):
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

In [13]:
y_true = tf.random.uniform([5], maxval=10, dtype=tf.int32)
y_pred = tf.random.uniform([5], maxval=10, dtype=tf.int32)
print(y_true)
print(y_pred)

tf.Tensor([1 3 1 3 4], shape=(5,), dtype=int32)
tf.Tensor([8 4 5 0 9], shape=(5,), dtype=int32)


In [14]:
get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=20>

To verify that your Function's graph is doing the same computation as its equivalent Python function, you can make it execute eagerly with tf.config.run_functions_eagerly(True). This is a switch that turns off Function's ability to create and run graphs, instead executing the code normally.

In [18]:
tf.config.run_functions_eagerly(True)

get_MSE(y_true, y_pred)

<tf.Tensor: shape=(), dtype=int32, numpy=20>

In [19]:
# Don't forget to set it back when you are done.
tf.config.run_functions_eagerly(False)

In [20]:
@tf.function
def get_MSE(y_true, y_pred):
  print("Calculating MSE!")
  sq_diff = tf.pow(y_true - y_pred, 2)
  return tf.reduce_mean(sq_diff)

In [21]:
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

Calculating MSE!


Is the output surprising? get_MSE only printed once even though it was called three times.

To explain, the print statement is executed when Function runs the original code in order to create the graph in a process known as "tracing". Tracing captures the TensorFlow operations into a graph, and print is not captured in the graph. That graph is then executed for all three calls without ever running the Python code again.

As a sanity check, let's turn off graph execution to compare:

In [22]:
# Now, globally set everything to run eagerly to force eager execution.
tf.config.run_functions_eagerly(True)

In [23]:
# Observe what is printed below.
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)
error = get_MSE(y_true, y_pred)

Calculating MSE!
Calculating MSE!
Calculating MSE!


### Toggle between eager and graph execution

In [24]:
tf.config.run_functions_eagerly(False)

print is a Python side effect, and there are other differences that you should be aware of when converting a function into a Function.

If you would like to print values in both eager and graph execution, use tf.print instead.

In [31]:
tf.config.run_functions_eagerly(True)

In [32]:
def unused_return_eager(x):
  # Get index 1 will fail when `len(x) == 1`
  tf.gather(x, [1]) # unused 
  return x

try:
  print(unused_return_eager(tf.constant([0.0])))
except tf.errors.InvalidArgumentError as e:
  # All operations are run during eager execution so an error is raised.
  print(f'{type(e).__name__}: {e}')

tf.Tensor([0.], shape=(1,), dtype=float32)


In [33]:
tf.config.run_functions_eagerly(False)

In [34]:
@tf.function
def unused_return_graph(x):
  tf.gather(x, [1]) # unused
  return x

# Only needed operations are run during graph exection. The error is not raised.
print(unused_return_graph(tf.constant([0.0])))

tf.Tensor([0.], shape=(1,), dtype=float32)


## Timing execution

In [35]:
x = tf.random.uniform(shape=[10, 10], minval=-1, maxval=2, dtype=tf.dtypes.int32)

def power(x, y):
  result = tf.eye(10, dtype=tf.dtypes.int32)
  for _ in range(y):
    result = tf.matmul(x, result)
  return result

print("Eager execution:", timeit.timeit(lambda: power(x, 100), number=1000))

Eager execution: 2.803953739999997


In [36]:
power_as_graph = tf.function(power)
print("Graph execution:", timeit.timeit(lambda: power_as_graph(x, 100), number=1000))

Graph execution: 0.4456176969997614


### To see tracing use a print

In [38]:
@tf.function
def a_function_with_python_side_effect(x):
  print("Tracing!") # An eager-only side effect.
  return x * x + tf.constant(2)

# This is traced the first time.
print(a_function_with_python_side_effect(tf.constant(2)))
# The second time through, you won't see the side effect.
print(a_function_with_python_side_effect(tf.constant(3)))

Tracing!
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(11, shape=(), dtype=int32)


In [39]:
# This retraces each time the Python argument changes,
# as a Python argument could be an epoch count or other
# hyperparameter.
print(a_function_with_python_side_effect(2))
print(a_function_with_python_side_effect(3))

Tracing!
tf.Tensor(6, shape=(), dtype=int32)
Tracing!
tf.Tensor(11, shape=(), dtype=int32)


## BETTER PERFORMANCE WITH TF.FUNCTION

https://www.tensorflow.org/guide/function

A Function you define (for example by applying the @tf.function decorator) is just like a core TensorFlow operation: You can execute it eagerly; you can compute gradients; and so on.

In [40]:
@tf.function  # The decorator converts `add` into a `Function`.
def add(a, b):
  return a + b

add(tf.ones([2, 2]), tf.ones([2, 2]))  #  [[2., 2.], [2., 2.]]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 2.],
       [2., 2.]], dtype=float32)>

In [41]:
v = tf.Variable(1.0)
with tf.GradientTape() as tape:
  result = add(v, 1.0)
tape.gradient(result, v)

<tf.Tensor: shape=(), dtype=float32, numpy=1.0>

You can use Functions inside other Functions.

In [42]:
@tf.function
def dense_layer(x, w, b):
  return add(tf.matmul(x, w), b)

dense_layer(tf.ones([3, 2]), tf.ones([2, 2]), tf.ones([2]))

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[3., 3.],
       [3., 3.],
       [3., 3.]], dtype=float32)>

Functions can be faster than eager code, especially for graphs with many small ops. But for graphs with a few expensive ops (like convolutions), you may not see much speedup.

In [43]:
import timeit
conv_layer = tf.keras.layers.Conv2D(100, 3)

@tf.function
def conv_fn(image):
  return conv_layer(image)

image = tf.zeros([1, 200, 200, 100])
# Warm up
conv_layer(image); conv_fn(image)
print("Eager conv:", timeit.timeit(lambda: conv_layer(image), number=10))
print("Function conv:", timeit.timeit(lambda: conv_fn(image), number=10))
print("Note how there's not much difference in performance for convolutions")

Eager conv: 0.0036601010006052093
Function conv: 0.004009455999948841
Note how there's not much difference in performance for convolutions


 For instance, Python supports polymorphism, but tf.Graph requires its inputs to have a specified data type and dimension. Or you may perform side tasks like reading command-line arguments, raising an error, or working with a more complex Python object; none of these things can run in a tf.Graph.

Function bridges this gap by separating your code in two stages:

1) In the first stage, referred to as "tracing", Function creates a new tf.Graph. Python code runs normally, but all TensorFlow operations (like adding two Tensors) are deferred: they are captured by the tf.Graph and not run.

2) In the second stage, a tf.Graph which contains everything that was deferred in the first stage is run. This stage is much faster than the tracing stage.

When Function does decide to trace, the tracing stage is immediately followed by the second stage, so calling the Function both creates and runs the tf.Graph

When we pass arguments of different types into a Function, both stages are run:

In [44]:
@tf.function
def double(a):
    print("Tracing with", a)
    return a + a

print(double(tf.constant(1)))
print()
print(double(tf.constant(1.1)))
print()
print(double(tf.constant("a")))
print()

Tracing with Tensor("a:0", shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)

Tracing with Tensor("a:0", shape=(), dtype=float32)
tf.Tensor(2.2, shape=(), dtype=float32)

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'aa', shape=(), dtype=string)



In [45]:
#You can use pretty_printed_concrete_signatures() to see all of the available traces:
print(double.pretty_printed_concrete_signatures())

double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()

double(a)
  Args:
    a: float32 Tensor, shape=()
  Returns:
    float32 Tensor, shape=()

double(a)
  Args:
    a: int32 Tensor, shape=()
  Returns:
    int32 Tensor, shape=()


### Obtaining concrete functions
Every time a function is traced, a new concrete function is created. You can directly obtain a concrete function, by using get_concrete_function.

In [47]:
print("Obtaining concrete trace")
double_strings = double.get_concrete_function(tf.constant("a"))
print("Executing traced function")
print(double_strings(tf.constant("a")))
print(double_strings(a=tf.constant("b")))

Obtaining concrete trace
Executing traced function
tf.Tensor(b'aa', shape=(), dtype=string)
tf.Tensor(b'bb', shape=(), dtype=string)


In [48]:
print(double_strings)

ConcreteFunction double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


In [49]:
print(double_strings.structured_input_signature)

((TensorSpec(shape=(), dtype=tf.string, name='a'),), {})


In [50]:
print(double_strings.structured_outputs)

Tensor("Identity:0", shape=(), dtype=string)


In [51]:
double_strings(tf.constant(1))

InvalidArgumentError: cannot compute __inference_double_109480 as input #0(zero-based) was expected to be a string tensor but is a int32 tensor [Op:__inference_double_109480]

In [52]:
# You can also call get_concrete_function on an InputSpec
double_strings_from_inputspec = double.get_concrete_function(tf.TensorSpec(shape=[], dtype=tf.string))
print(double_strings_from_inputspec(tf.constant("c")))

Tracing with Tensor("a:0", shape=(), dtype=string)
tf.Tensor(b'cc', shape=(), dtype=string)


In [54]:
print(double_strings_from_inputspec)

ConcreteFunction double(a)
  Args:
    a: string Tensor, shape=()
  Returns:
    string Tensor, shape=()


#### Python arguments are given special treatment in a concrete function's input signature

In [55]:
@tf.function
def pow(a, b):
  return a ** b

square = pow.get_concrete_function(a=tf.TensorSpec(None, tf.float32), b=2)
print(square)

ConcreteFunction pow(a, b=2)
  Args:
    a: float32 Tensor, shape=<unknown>
  Returns:
    float32 Tensor, shape=<unknown>


In [56]:
square(tf.constant(10.0))

<tf.Tensor: shape=(), dtype=float32, numpy=100.0>

In [57]:
square(tf.constant(10.0), b=3)

TypeError: ConcreteFunction pow(a, b) was constructed with int value 2 in b, but was called with int value 3

#### Obtain graphs

In [58]:
graph = double_strings.graph
for node in graph.as_graph_def().node:
  print(f'{node.input} -> {node.name}')

[] -> a
['a', 'a'] -> add
['add'] -> Identity


## AutoGraph transformations

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/control_flow.md#if-statements

https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/autograph/g3doc/reference/control_flow.md#while-statements

AutoGraph is a library that is on by default in tf.function, and transforms a subset of Python eager code into graph-compatible TensorFlow ops. This includes control flow like if, for, while.

TensorFlow ops like tf.cond and tf.while_loop continue to work, but control flow is often easier to write and understand when written in Python

#### AutoGraph will convert some if \<condition\> statements into the equivalent tf.cond calls. This substitution is made if \<condition\> is a Tensor. Otherwise, the if statement is executed as a Python conditional.
    
A Python conditional executes during tracing, so exactly one branch of the conditional will be added to the graph. Without AutoGraph, this traced graph would be unable to take the alternate branch if there is data-dependent control flow.

**tf.cond traces and adds both branches of the conditional to the graph, dynamically selecting a branch at execution time.**

#### AutoGraph will convert some for and while statements into the equivalent TensorFlow looping ops, like tf.while_loop. If not converted, the for or while loop is executed as a Python loop.

- for x in y: if y is a Tensor, convert to tf.while_loop. In the special case where y is a tf.data.Dataset, a combination of tf.data.Dataset ops are generated.
- while <condition>: if <condition> is a Tensor, convert to tf.while_loop
    
A Python loop executes during tracing, adding additional ops to the tf.Graph for every iteration of the loop.

A TensorFlow loop traces the body of the loop, and dynamically selects how many iterations to run at execution time. The loop body only appears once in the generated tf.Graph.

### A common pitfall is to loop over Python/NumPy data within a tf.function. This loop will execute during the tracing process, adding a copy of your model to the tf.Graph for each iteration of the loop.

### Accumulating values in a loop
A common pattern is to accumulate intermediate values from a loop. Normally, this is accomplished by appending to a Python list or adding entries to a Python dictionary. However, as these are Python side effects, they will not work as expected in a dynamically unrolled loop. Use tf.TensorArray to accumulate results from a dynamically unrolled loop

In [ ]:
batch_size = 2
seq_len = 3
feature_size = 4

def rnn_step(inp, state):
  return inp + state

@tf.function
def dynamic_rnn(rnn_step, input_data, initial_state):
  # [batch, time, features] -> [time, batch, features]
  input_data = tf.transpose(input_data, [1, 0, 2])
  max_seq_len = input_data.shape[0]

  states = tf.TensorArray(tf.float32, size=max_seq_len)
  state = initial_state
  for i in tf.range(max_seq_len):
    state = rnn_step(input_data[i], state)
    states = states.write(i, state)
  return tf.transpose(states.stack(), [1, 0, 2])

dynamic_rnn(rnn_step,
            tf.random.uniform([batch_size, seq_len, feature_size]),
            tf.zeros([batch_size, feature_size]))

TensorFlow Function has a few limitations by design that you should be aware of when converting a Python function to a Function.

#### Side effects, like printing, appending to lists, and mutating globals, can behave unexpectedly inside a Function, sometimes executing twice or not all. They only happen the first time you call a Function with a set of inputs. Afterwards, the traced tf.Graph is reexecuted, without executing the Python code.
The general rule of thumb is to avoid relying on Python side effects in your logic and only use them to debug your traces. Otherwise, TensorFlow APIs like tf.data, tf.print, tf.summary, tf.Variable.assign, and tf.TensorArray are the best way to ensure your code will be executed by the TensorFlow runtime with each call.

### Changing Python global and free variables
Changing Python global and free variables counts as a Python side effect, so it only happens during tracing.

In [59]:
external_list = []

@tf.function
def side_effect(x):
  print('Python side effect')
  external_list.append(x)

side_effect(1)
side_effect(1)
side_effect(1)
# The list append only happened once!
assert len(external_list) == 1

Python side effect


In [60]:
external_list

[1]

You should avoid mutating containers like lists, dicts, other objects that live outside the Function. Instead, use arguments and TF objects. For example, the section "Accumulating values in a loop" has one example of how list-like operations can be implemented

### Using Python iterators and generators
Many Python features, such as generators and iterators, rely on the Python runtime to keep track of state. In general, while these constructs work as expected in eager mode, they are examples of Python side effects and therefore only happen during tracing.

In [61]:
@tf.function
def buggy_consume_next(iterator):
  tf.print("Value:", next(iterator))

iterator = iter([1, 2, 3])
buggy_consume_next(iterator)
# This reuses the first value from the iterator, rather than consuming the next value.
buggy_consume_next(iterator)
buggy_consume_next(iterator)

Value: 1
Value: 1
Value: 1


Just like how TensorFlow has a specialized tf.TensorArray for list constructs, it has a specialized tf.data.Iterator for iteration constructs. See the section on AutoGraph transformations for an overview. Also, the tf.data API can help implement generator patterns:

In [ ]:
@tf.function
def good_consume_next(iterator):
  # This is ok, iterator is a tf.data.Iterator
  tf.print("Value:", next(iterator))

ds = tf.data.Dataset.from_tensor_slices([1, 2, 3])
iterator = iter(ds)
good_consume_next(iterator)
good_consume_next(iterator)
good_consume_next(iterator)

### DON't understand Deleting tf.Variables between Function calls
https://www.tensorflow.org/guide/function#deleting_tfvariables_between_function_calls

### DON't understand All outputs of a tf.function must be return values
https://www.tensorflow.org/guide/function#all_outputs_of_a_tffunction_must_be_return_values

###

### All outputs of a tf.function must be return values

With the exception of tf.Variables, a tf.function must return all its outputs. Attempting to directly access any tensors from a function without going through return values causes "leaks".

In [ ]:
x = None

@tf.function
def leaky_function(a):
  global x
  x = a + 1  # Bad - leaks local tensor
  return a + 2

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns
with assert_raises(AttributeError):
  x.numpy()  # Bad - tensor leaked from inside the function, cannot be used here
print(x)

In [ ]:
@tf.function
def leaky_function(a):
  global x
  x = a + 1  # Bad - leaks local tensor
  return x  # Good - uses local tensor

correct_a = leaky_function(tf.constant(1))

print(correct_a.numpy())  # Good - value obtained from function's returns
with assert_raises(AttributeError):
  x.numpy()  # Bad - tensor leaked from inside the function, cannot be used here
print(x)

@tf.function
def captures_leaked_tensor(b):
  b += x  # Bad - `x` is leaked from `leaky_function`
  return b

with assert_raises(TypeError):
  captures_leaked_tensor(tf.constant(2))

In [ ]:
class MyClass:

  def __init__(self):
    self.field = None

external_list = []
external_object = MyClass()

def leaky_function():
  a = tf.constant(1)
  external_list.append(a)  # Bad - leaks tensor
  external_object.field = a  # Bad - leaks tensor

#### Depending on Python global and free variables
Function creates a new ConcreteFunction when called with a new value of a Python argument. However, it does not do that for the Python closure, globals, or nonlocals of that Function. If their value changes in between calls to the Function, the Function will still use the values they had when it was traced. This is different from how regular Python functions work.

For that reason, we recommend a functional programming style that uses arguments instead of closing over outer names.

In [62]:
@tf.function
def buggy_add():
  return 1 + foo

@tf.function
def recommended_add(foo):
  return 1 + foo

foo = 1
print("Buggy:", buggy_add())
print("Correct:", recommended_add(foo))

Buggy: tf.Tensor(2, shape=(), dtype=int32)
Correct: tf.Tensor(2, shape=(), dtype=int32)


In [63]:
print("Updating the value of `foo` to 100!")
foo = 100
print("Buggy:", buggy_add())  # Did not change!
print("Correct:", recommended_add(foo))

Updating the value of `foo` to 100!
Buggy: tf.Tensor(2, shape=(), dtype=int32)
Correct: tf.Tensor(101, shape=(), dtype=int32)


The recommendation to pass Python objects as arguments into tf.function has a number of known issues, that are expected to be fixed in the future. In general, you can rely on consistent tracing if you use a Python primitive or tf.nest-compatible structure as an argument or pass in a different instance of an object into a Function. However, Function will not create a new trace when you pass the same object and only change its attributes

In [78]:
class SimpleModel(tf.Module):
    def __init__(self):
        # These values are *not* tf.Variables.
        self.bias = 0.
        self.weight = 2.

@tf.function
def evaluate(model, x):
    print('Tracing!')
    return model.weight * x + model.bias

simple_model = SimpleModel()
x = tf.constant(10.)
print(evaluate(simple_model, x))
x = tf.constant(11.)
print(evaluate(simple_model, x))
x = tf.constant(12.)
print(evaluate(simple_model, x))

Tracing!
tf.Tensor(20.0, shape=(), dtype=float32)
tf.Tensor(22.0, shape=(), dtype=float32)
tf.Tensor(24.0, shape=(), dtype=float32)


In [79]:
hex(id(simple_model))

'0x1867a67f940'

In [80]:
print("Adding bias!")
simple_model.bias += 5.0
print(evaluate(simple_model, x))  # Didn't change :(

Adding bias!
tf.Tensor(24.0, shape=(), dtype=float32)


In [81]:
hex(id(simple_model))

'0x1867a67f940'

Using the same Function to evaluate the updated instance of the model will be buggy since the updated model has the same cache key as the original model.

For that reason, we recommend that you write your Function to avoid depending on mutable object attributes or create new objects.

If that is not possible, **one workaround is to make new Functions each time you modify your object to force retracing:**

In [84]:
def evaluate(model, x):
  return model.weight * x + model.bias

new_model = SimpleModel()
evaluate_no_bias = tf.function(evaluate).get_concrete_function(new_model, x)
# Don't pass in `new_model`, `Function` already captured its state during tracing.
print(evaluate_no_bias(x))

tf.Tensor(24.0, shape=(), dtype=float32)


## I DON'T Understand why we do not need to pass new_model in the evaluate_with_bias....

In [85]:
print("Adding bias!")
new_model.bias += 5.0
# Create new Function and ConcreteFunction since you modified new_model.
evaluate_with_bias = tf.function(evaluate).get_concrete_function(new_model, x)
print(evaluate_with_bias(x)) # Don't pass in `new_model`.

Adding bias!
tf.Tensor(29.0, shape=(), dtype=float32)


As retracing can be expensive, you can use tf.Variables as object attributes, which can be mutated (but not changed, careful!) for a similar effect without needing a retrace.

In [86]:
class BetterModel:

  def __init__(self):
    self.bias = tf.Variable(0.)
    self.weight = tf.Variable(2.)

@tf.function
def evaluate(model, x):
  return model.weight * x + model.bias

better_model = BetterModel()
print(evaluate(better_model, x))

tf.Tensor(24.0, shape=(), dtype=float32)


In [87]:
print("Adding bias!")
better_model.bias.assign_add(5.0)  # Note: instead of better_model.bias += 5
print(evaluate(better_model, x))  # This works!

Adding bias!
tf.Tensor(29.0, shape=(), dtype=float32)


Function only supports singleton tf.Variables created once on the first call, and reused across subsequent function calls. The code snippet below would create a new tf.Variable in every function call, which results in a ValueError exception.

In [88]:
@tf.function
def f(x):
  v = tf.Variable(1.0)
  return v


f(1.0)

ValueError: in user code:

    C:\Users\SiFuBrO\AppData\Local\Temp/ipykernel_6444/135690660.py:3 f  *
        v = tf.Variable(1.0)
    c:\users\sifubro\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\variables.py:262 __call__  **
        return cls._variable_v2_call(*args, **kwargs)
    c:\users\sifubro\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\variables.py:244 _variable_v2_call
        return previous_getter(
    c:\users\sifubro\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\ops\variables.py:67 getter
        return captured_getter(captured_previous, **kwargs)
    c:\users\sifubro\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\eager\def_function.py:730 invalid_creator_scope
        raise ValueError(

    ValueError: tf.function-decorated function tried to create variables on non-first call.


A common pattern used to work around this limitation is to start with a Python None value, then conditionally create the tf.Variable if the value is None:

In [ ]:
class Count(tf.Module):
  def __init__(self):
    self.count = None

  @tf.function
  def __call__(self):
    if self.count is None:
      self.count = tf.Variable(0)
    return self.count.assign_add(1)

c = Count()
print(c())
print(c())

### ValueError: tf.function only supports singleton tf.Variables created on the first call.

#### Using with multiple Keras models
You may also encounter ValueError: tf.function only supports singleton tf.Variables created on the first call. when passing different model instances to the same Function.

This error occurs because Keras models (which do not have their input shape defined) and Keras layers create tf.Variabless when they are first called. You may be attempting to initialize those variables inside a Function, which has already been called. To avoid this error, try calling model.build(input_shape) to initialize all the weights before training the model.

### TIPS for @tf.function

https://www.tensorflow.org/guide/intro_to_graphs#tffunction_best_practices

- Debug in eager mode, then decorate with @tf.function
- Toggle between eager and graph execution early and often with tf.config.run_functions_eagerly to pinpoint if/ when the two modes diverge.
- Create tf.Variables outside the Python function and modify them on the inside. The same goes for objects that use tf.Variable, like keras.layers, keras.Models and tf.optimizers.
- Avoid writing functions that depend on `outer Python variables` (https://www.tensorflow.org/guide/function#depending_on_python_global_and_free_variables), excluding tf.Variables and Keras objects.
- Prefer to write functions which take tensors and other TensorFlow types as input. You can pass in other object types (https://www.tensorflow.org/guide/function#depending_on_python_objects) but be careful!
- Don't rely on Python side effects like object mutation or list appends
- tf.function works best with TensorFlow ops; NumPy and Python calls are converted to constants.
- Include as much computation as possible under a tf.function to maximize the performance gain. For example, decorate a whole training step or the entire training loop.
- `tf.function` is commonly used to speed up training loops, and you can learn more about it in Writing a training loop from scratch with Keras. (https://www.tensorflow.org/guide/keras/writing_a_training_loop_from_scratch#speeding_up_your_training_step_with_tffunction)
-  how to set input specifications and use tensor arguments to avoid retracing (https://www.tensorflow.org/guide/function#controlling_retracing)
- Specify input_signature in tf.function to limit tracing.
- Specify a [None] dimension in tf.TensorSpec to allow for flexibility in trace reuse. Since TensorFlow matches tensors based on their shape, using a None dimension as a wildcard will allow Functions to reuse traces for variably-sized input.
- Cast Python arguments to Tensors to reduce retracing
- If you need to force retracing, create a new Function. Separate Function objects are guaranteed not to share traces.
- tf.debugging.enable_check_numerics is an easy way to track down where NaNs and Inf are created
- AutoGraph will convert some if \<condition\> statements into the equivalent tf.cond calls. This substitution is made if \<condition\> is a Tensor. Otherwise, the if statement is executed as a Python conditional.
- for x in y: if y is a Tensor, convert to tf.while_loop. In the special case where y is a tf.data.Dataset, a combination of tf.data.Dataset ops are generated.
- while \<condition\>: if \<condition\> is a Tensor, convert to tf.while_loop.
- A common pitfall is to loop over Python/NumPy data within a tf.function. This loop will execute during the tracing process, adding a copy of your model to the tf.Graph for each iteration of the loop. If you want to wrap the entire training loop in tf.function, the safest way to do this is to wrap your data as a tf.data.Dataset so that AutoGraph will dynamically unroll the training loop.
- A common pattern is to accumulate intermediate values from a loop. Normally, this is accomplished by appending to a Python list or adding entries to a Python dictionary. However, as these are Python side effects, they will not work as expected in a dynamically unrolled loop. Use tf.TensorArray to accumulate results from a dynamically unrolled loop.    
- Side effects, like printing, appending to lists, and mutating globals, can behave unexpectedly inside a Function, sometimes executing twice or not all. They only happen the first time you call a Function with a set of inputs. Afterwards, the traced tf.Graph is reexecuted, without executing the Python code.
- The general rule of thumb is to avoid relying on Python side effects in your logic and only use them to debug your traces. Otherwise, TensorFlow APIs like tf.data, tf.print, tf.summary, tf.Variable.assign, and tf.TensorArray are the best way to ensure your code will be executed by the TensorFlow runtime with each call.
- **If you would like to execute Python code during each invocation of a Function, tf.py_function is an exit hatch. The drawback of tf.py_function is that it's not portable or particularly performant, cannot be saved with SavedModel, and does not work well in distributed (multi-GPU, TPU) setups. Also, since tf.py_function has to be wired into the graph, it casts all inputs/outputs to tensors.**
- Changing Python global and free variables counts as a Python side effect, so it only happens during tracing.
- You should avoid mutating containers like lists, dicts, other objects that live outside the Function. Instead, use arguments and TF objects. For example, the section "Accumulating values in a loop" has one example of how list-like operations can be implemented (with tf.TensorArray)
- Many Python features, such as generators and iterators, rely on the Python runtime to keep track of state. In general, while these constructs work as expected in eager mode, they are examples of Python side effects and therefore only happen during tracing.
- Just like how TensorFlow has a specialized tf.TensorArray for list constructs, it has a specialized tf.data.Iterator for iteration constructs. See the section on AutoGraph transformations for an overview. Also, the tf.data API can help implement generator patterns:
- not use global inside tf.function to access variables outside. Just pass them as arguments (tensors????)
- Do NOT mutate an external Python collection or an object from tf.function
- Function creates a new ConcreteFunction when called with a new value of a Python argument. However, it does not do that for the Python closure, globals, or nonlocals of that Function. If their value changes in between calls to the Function, the Function will still use the values they had when it was traced. This is different from how regular Python functions work. For that reason, we recommend a functional programming style that uses arguments instead of closing over outer names.
- The recommendation to pass Python objects as arguments into tf.function has a number of known issues, that are expected to be fixed in the future. In general, you can rely on consistent tracing if you use a Python primitive or tf.nest-compatible structure as an argument or pass in a different instance of an object into a Function. However, Function will not create a new trace when you pass the same object and only change its attributes
- we recommend that you write your Function to avoid depending on mutable object attributes or create new objects.If that is not possible, one workaround is to make new Functions each time you modify your object to force retracing
- Function only supports singleton tf.Variables created once on the first call, and reused across subsequent function calls. The code snippet below would create a new tf.Variable in every function call, which results in a ValueError exception.A common pattern used to work around this limitation is to start with a Python None value, then conditionally create the tf.Variable if the value is None (https://www.tensorflow.org/guide/function#creating_tfvariables)
- 




**RULES OS TRACING**
- https://www.tensorflow.org/guide/function#rules_of_tracing

see also

- https://stackoverflow.com/questions/59847045/should-i-use-tf-function-for-all-functions

- A tf.Graph is the raw, language-agnostic, portable representation of a TensorFlow computation.
- A ConcreteFunction wraps a tf.Graph.
- A Function manages a cache of ConcreteFunctions and picks the right one for your inputs.
- tf.function wraps a Python function, returning a Function object.
- Tracing creates a tf.Graph and wraps it in a ConcreteFunction, also known as a trace.

### !TODO! See Better performance with tf.function